In [1]:
import os

import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from multiprocessing import Pool

In [2]:
from PIL import Image
import cv2

fast = cv2.FastFeatureDetector_create()
sift = cv2.xfeatures2d.SIFT_create()
def keyp(img):
    try:            
        # find and draw the keypoints
        kp = fast.detect(img,None)
        kp =len(kp)
        return kp
    except:
        return 0

def sift(img):
    try:
        kp = sift.detect(img,None)
        kp = len(kp)
        return kp
    except:
        return 0
    

In [3]:
images_dir_tr = '../input/train_jpg/'
images_dir_te = '../input/test_jpg/'

In [4]:
from tqdm import tqdm

image_files_tr = [x.path for x in tqdm(os.scandir(images_dir_tr))]
image_files_te = [x.path for x in tqdm(os.scandir(images_dir_te))]
image_files = image_files_tr+image_files_te

1390832it [00:12, 115737.09it/s]
465829it [00:05, 90555.34it/s]


In [16]:
from collections import Counter
from pprint import pprint

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

def classify_and_plot(img,cv_img,cv_img_rgb,image_path):
    """Classify an image with different models.
    Plot it and its predicitons.
    """
    kp = keyp(cv_img)
    sf = sift(cv_img)
    img_id = image_path[(image_path.rfind('/')+1):-4]
    return (img, cv_img, cv_img_rgb, kp, sf, img_id)

def get_data_from_image(dat):
    img_size = [dat[0].size[0], dat[0].size[1]]
    (means, stds) = cv2.meanStdDev(dat[2])
    mean_color = np.mean(dat[2].flatten())
    std_color = np.std(dat[2].flatten())
    color_stats = np.concatenate([means, stds]).flatten()
    
    #print(len(color_stats.tolist()))
    df = pd.DataFrame([img_size+ [mean_color] + [std_color] + color_stats.tolist() +[dat[3]]+[dat[4]]+[dat[5]]],
                      columns = ['img_size_x', 'img_size_y', 'img_mean_color', 'img_std_color', 'img_blue_mean', 
                                 'img_green_mean', 'img_red_mean', 'img_blue_std', 'image_green_std', 
                                 'image_red_std', 'kp','sf','image'])
    return df



In [17]:
def process(img_path):
    img = Image.open(img_path)
    cv_img = cv2.imread(img_path,0)
    cv_img_rgb = cv2.imread(img_path)
    dat = classify_and_plot(img, cv_img,cv_img_rgb,img_path)
    #print(img_path)
    df = get_data_from_image(dat)
    return df

In [19]:
with Pool(8) as p:
    df_list = list(tqdm_notebook(p.imap(process,image_files),total=len(image_files)))

A Jupyter Widget

In [21]:
len(df_list)

1856661

In [32]:
for i in range(186):
    print(i)
    idx = i*10000
    pd.concat(df_list[idx:(idx+10000)],axis=0).to_csv('../features-0625/img_fe_'+str(i)+'.csv',index=None)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185


In [33]:
fes = os.listdir('../features-0625/')

In [35]:
df_chunks = []
for file in tqdm(fes):
    df_chunks.append(pd.read_csv('../features-0625/'+file))

100%|██████████| 186/186 [00:08<00:00, 21.89it/s]


In [36]:
df_final = pd.concat(df_chunks,axis=0)

In [40]:
df_final.to_csv('img_features_keypoint.csv',index=None)